In [102]:
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter, OrderedDict
import json
import momi

SyntaxError: invalid syntax (demography.py, line 369)

## Read the dadi sfs


In [103]:
def dadi_to_momi(infile, outfile=None, verbose=False):
    if outfile == None:
        outfile = infile.split(".sfs")[0] + "_momi.sfs"
    dat = open(infile).readlines()
    ## Get rid of comment lines
    dat = [x.strip() for x in dat if not x.startswith("#")]
    if not len(dat) == 3:
        raise Exception("Malformed dadi sfs {}.\n  Must have 3 lines, yours has {}".format(infile, len(dat)))

    ## Parse the info line into nsamps per pop (list of ints), folding flag, and pop names list (if they are given)
    info = dat[0].split()
    nsamps = []
    ## Keep carving off values as long as they cast successfully as int
    for i in info:
        try:
            nsamps.append(int(i))
        except:
            pass
    nsamps = np.array(nsamps)
    pops = [x.replace('"', '') for x in info[len(nsamps)+1:]]
    folded = info[len(nsamps)]
    folded = False if "un" in folded else True
    if not len(pops) == len(nsamps):
        print("Number of populations doesn't agree with number of pop names, using generic names.")
        pops = ["pop"+x for x in range(len(nsamps))]
    if verbose: print("Info nsamps={} folded={} pops={}".format(nsamps, folded, pops))

    ## Get mask
    mask = list(map(int, dat[2].split()))
    if verbose: print(mask)

    ## Get sfs, and reshape based on sample sizes
    sfs = list(map(float, dat[1].split()))
    if verbose: print(sfs)
    length = np.ma.array(sfs, mask=mask).sum()
    sfs = np.array(sfs).reshape(nsamps)
    if verbose: print("length {}".format(length))
    if verbose: print(sfs)

    ## Get counts per sfs bin
    counts = Counter()
    for sfsbin in product(*[range(y) for y in [x for x in nsamps]]):
        ## Ignore monomorphic snps
        ## nsamps minus 1 here because of the off by one diff between number
        ## of bins in the sfs and actual number of samples
        if sfsbin == tuple(nsamps-1) or sfsbin == tuple([0] * len(nsamps)):
            continue
        ## ignore zero bin counts
        if sfs[sfsbin] == 0:
            continue
        if verbose: print(sfsbin, sfs[sfsbin]),
        counts.update({sfsbin:sfs[sfsbin]})
    if verbose: print("nbins {}".format(len(counts)))

    ## Convert SFS bin style into momi config style
    configs = pd.DataFrame(index=range(len(counts)), columns=pops)
    
    locus_info = []
    for i, c in enumerate(counts):
        ## (n-1) here because nbins in dadi sfs is n+1
        cfg = np.array([[(n-1)-x, x] for x,n in zip(c, nsamps)])
        configs.iloc[i] = [list(map(int, list(x))) for x in cfg]
        locus_info.append([0, i, counts[c]])
    if verbose: print("n_snps {}".format(np.sum([x[2] for x in locus_info])))

    ## Finally build the momi style sfs dictionary and write it out
    momi_sfs = {"sampled_pops":pops,
        "folded":folded,
        "length":int(length),
        "configs":configs.values.tolist(),
        "(locus,config_id,count)":locus_info}

    with open(outfile, 'w') as out:
        out.write("{}".format(json.dumps(momi_sfs)))
    ## make it pretty
    sfs = momi.Sfs.load(outfile)
    ## Fold if unfolded
    if folded: sfs = sfs.fold()
    sfs.dump(outfile)
#dadi_to_momi(infile, verbose=True)
dadi_to_momi("/home/isaac/easySFS/jupyter-notebooks/1000_folded/dadi/pop1-pop2.sfs", verbose=False)
dadi_to_momi("/home/isaac/easySFS/jupyter-notebooks/1000_unfolded/dadi/pop1-pop2.sfs", verbose=False)

NameError: global name 'momi' is not defined

In [78]:
d = OrderedDict([('[[7 1]\n [5 3]]', 2), ('[[5 3]\n [8 0]]', 50), ('[[8 0]\n [0 8]]', 74), ('[[8 0]\n [1 7]]', 18), ('[[6 2]\n [2 6]]', 2), ('[[7 1]\n [2 6]]', 1), ('[[5 3]\n [1 7]]', 1), ('[[6 2]\n [3 5]]', 2), ('[[8 0]\n [5 3]]', 48), ('[[4 4]\n [8 0]]', 32), ('[[5 3]\n [5 3]]', 1), ('[[8 0]\n [2 6]]', 23), ('[[7 1]\n [3 5]]', 4), ('[[5 3]\n [0 8]]', 38), ('[[6 2]\n [6 2]]', 1), ('[[5 3]\n [3 5]]', 2), ('[[4 4]\n [7 1]]', 2), ('[[7 1]\n [7 1]]', 2), ('[[8 0]\n [4 4]]', 32), ('[[6 2]\n [0 8]]', 23), ('[[7 1]\n [4 4]]', 3), ('[[8 0]\n [3 5]]', 28), ('[[6 2]\n [7 1]]', 1), ('[[4 4]\n [6 2]]', 1), ('[[7 1]\n [8 0]]', 185), ('[[8 0]\n [7 1]]', 185), ('[[7 1]\n [0 8]]', 17), ('[[5 3]\n [2 6]]', 2), ('[[6 2]\n [1 7]]', 4), ('[[8 0]\n [6 2]]', 66), ('[[6 2]\n [8 0]]', 87), ('[[5 3]\n [4 4]]', 2), ('[[6 2]\n [4 4]]', 2)])
for i, row in configs.iterrows():
    print(i, row.values)

(0, array([list([7, 1]), list([5, 3])], dtype=object))
(1, array([list([5, 3]), list([8, 0])], dtype=object))
(2, array([list([8, 0]), list([0, 8])], dtype=object))
(3, array([list([8, 0]), list([1, 7])], dtype=object))
(4, array([list([6, 2]), list([2, 6])], dtype=object))
(5, array([list([7, 1]), list([2, 6])], dtype=object))
(6, array([list([5, 3]), list([1, 7])], dtype=object))
(7, array([list([6, 2]), list([3, 5])], dtype=object))
(8, array([list([8, 0]), list([5, 3])], dtype=object))
(9, array([list([4, 4]), list([8, 0])], dtype=object))
(10, array([list([5, 3]), list([5, 3])], dtype=object))
(11, array([list([8, 0]), list([2, 6])], dtype=object))
(12, array([list([7, 1]), list([3, 5])], dtype=object))
(13, array([list([5, 3]), list([0, 8])], dtype=object))
(14, array([list([6, 2]), list([6, 2])], dtype=object))
(15, array([list([5, 3]), list([3, 5])], dtype=object))
(16, array([list([4, 4]), list([7, 1])], dtype=object))
(17, array([list([7, 1]), list([7, 1])], dtype=object))
(1

In [7]:
dat = open(infile).readlines()
## Get rid of comment lines
dat = [x.strip() for x in dat if not x.startswith("#")]
if not len(dat) == 3:
    print("Malformed dadi sfs. Must have 3 lines, yours has {}".format(len(dat)))
dat


['5 5 unfolded "nuttalli" "pugetensis"',
 '776.1460592424144 115.1222178069079 24.96239616420053 6.218025728002017 1.073392240447087 107.7264883754097 40.66889634511905 18.65179284371549 6.329399414025897 1.065189279571691 26.17961131785413 19.70143248690903 12.07926204967499 4.544540090166469 0.7711291771526296 6.954173763250236 7.96663510183704 5.391325472681568 2.026077574941607 0.3284864310491528 1.410459329911039 1.890674276442391 1.269396064688684 0.4543005907520786 0.06863883286683978',
 '1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1']

In [122]:
## Get nsamps
info = dat[0].split()
nsamps = []
for i in info:
    try:
        nsamps.append(int(i))
    except:
        pass
nsamps = np.array(nsamps)
pops = info[len(nsamps)+1:]
folded = info[len(nsamps)]
if not len(pops) == len(nsamps):
    print("Number of populations doesn't agree with number of pop names, using generic names.")
    pops = ["pop"+str(x) for x in range(len(nsamps))]
print(nsamps, folded, pops)

## Get mask
mask = map(int, dat[2].split())
print(mask)
if np.sum(mask) > 2:
    print("momi sfs conversion requires unfolded sfs. Folded sfs will 'work' but with unknown results.")

## Get sfs
sfs = map(int, map(np.round, map(float, dat[1].split())))
print(sfs)
length = np.ma.array(sfs, mask=mask).sum()
sfs = np.array(sfs).reshape(nsamps)
print("length {}".format(length))
print(sfs)


(array([5, 5]), 'unfolded', ['"nuttalli"', '"pugetensis"'])
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[776, 115, 25, 6, 1, 108, 41, 19, 6, 1, 26, 20, 12, 5, 1, 7, 8, 5, 2, 0, 1, 2, 1, 0, 0]
length 412
[[776 115  25   6   1]
 [108  41  19   6   1]
 [ 26  20  12   5   1]
 [  7   8   5   2   0]
 [  1   2   1   0   0]]


In [106]:
from collections import Counter
counts = Counter()
for sfsbin in product(*[range(y) for y in [x for x in nsamps]]):
    ## Ignore monomorphic snps
    ## nsamps minus 1 here because of the off by one diff between number
    ## of bins in the sfs and actual number of samples
    if sfsbin == tuple(nsamps-1) or sfsbin == tuple([0] * len(nsamps)):
        continue
    ## ignore zero bin counts
    if sfs[sfsbin] == 0:
        continue
    print(sfsbin, sfs[sfsbin])
    counts.update({sfsbin:sfs[sfsbin]})
counts
configs = pd.DataFrame(index=range(len(counts)), columns=pops)

#print(configs)


((0, 1), 115)
((0, 2), 25)
((0, 3), 6)
((0, 4), 1)
((1, 0), 108)
((1, 1), 41)
((1, 2), 19)
((1, 3), 6)
((1, 4), 1)
((2, 0), 26)
((2, 1), 20)
((2, 2), 12)
((2, 3), 5)
((2, 4), 1)
((3, 0), 7)
((3, 1), 8)
((3, 2), 5)
((3, 3), 2)
((4, 0), 1)
((4, 1), 2)
((4, 2), 1)


In [112]:
locus_count = 0
locus_info = []
for i, c in enumerate(counts):
    ## (n-1) here because nbins in dadi sfs is n+1
    configs.iloc[i] = [[(n-1)-x, x] for x,n in zip(c, nsamps)]
    locus_info.extend([x, i, 1] for x in range(locus_count, locus_count+counts[c]))
    locus_count += counts[c]
configs
locus_info

[[0, 0, 1],
 [1, 0, 1],
 [2, 0, 1],
 [3, 0, 1],
 [4, 0, 1],
 [5, 0, 1],
 [6, 0, 1],
 [7, 0, 1],
 [8, 0, 1],
 [9, 0, 1],
 [10, 0, 1],
 [11, 0, 1],
 [12, 0, 1],
 [13, 0, 1],
 [14, 0, 1],
 [15, 0, 1],
 [16, 0, 1],
 [17, 0, 1],
 [18, 0, 1],
 [19, 0, 1],
 [20, 0, 1],
 [21, 0, 1],
 [22, 0, 1],
 [23, 0, 1],
 [24, 0, 1],
 [25, 0, 1],
 [26, 0, 1],
 [27, 0, 1],
 [28, 0, 1],
 [29, 0, 1],
 [30, 0, 1],
 [31, 0, 1],
 [32, 0, 1],
 [33, 0, 1],
 [34, 0, 1],
 [35, 0, 1],
 [36, 0, 1],
 [37, 0, 1],
 [38, 0, 1],
 [39, 0, 1],
 [40, 0, 1],
 [41, 0, 1],
 [42, 0, 1],
 [43, 0, 1],
 [44, 0, 1],
 [45, 0, 1],
 [46, 0, 1],
 [47, 0, 1],
 [48, 0, 1],
 [49, 0, 1],
 [50, 0, 1],
 [51, 0, 1],
 [52, 0, 1],
 [53, 0, 1],
 [54, 0, 1],
 [55, 0, 1],
 [56, 0, 1],
 [57, 0, 1],
 [58, 0, 1],
 [59, 0, 1],
 [60, 0, 1],
 [61, 0, 1],
 [62, 0, 1],
 [63, 0, 1],
 [64, 0, 1],
 [65, 0, 1],
 [66, 0, 1],
 [67, 0, 1],
 [68, 0, 1],
 [69, 0, 1],
 [70, 0, 1],
 [71, 0, 1],
 [72, 0, 1],
 [73, 0, 1],
 [74, 0, 1],
 [75, 0, 1],
 [76, 0, 1],
 [77, 0, 

In [123]:
momi_sfs = {"sampled_pops":pops,
        "folded":folded,
        "length":length,
        "configs":configs.values.tolist(),
        "(locus,config_id,count)":locus_info}


In [124]:
with open("/tmp/watdo.sfs", 'w') as outfile:
    outfile.write("{}".format(momi_sfs))

## Crap below here

In [171]:
!pwd
!../easySFS.py -i 25loci.vcf -p simout/simpops.txt --proj 8,8 -o 25_folded -f
!../easySFS.py -i 25loci.vcf -p simout/simpops.txt --proj 8,8 -o 25_unfolded --unfolded -f
dadi_to_momi("./25_folded/dadi/pop1-pop2.sfs")
dadi_to_momi("./25_unfolded/dadi/pop1-pop2.sfs")


/home/isaac/easySFS/jupyter-notebooks
Processing 2 populations - ['pop1', 'pop2']
  Sampling one snp per locus
Doing 1D sfs - pop1
Doing 1D sfs - pop2
Doing 2D sfs - ('pop1', 'pop2')
Doing multiSFS for all pops
Processing 2 populations - ['pop1', 'pop2']
  Sampling one snp per locus
Doing 1D sfs - pop1
Doing 1D sfs - pop2
Doing 2D sfs - ('pop1', 'pop2')
Doing multiSFS for all pops
momi sfs conversion requires unfolded sfs. Folded sfs will 'work' but with unknown results.


In [172]:
!pwd
!../easySFS.py -i 100loci.vcf -p simout/simpops.txt --proj 8,8 -o 100_folded -f
!../easySFS.py -i 100loci.vcf -p simout/simpops.txt --proj 8,8 -o 100_unfolded --unfolded -f
dadi_to_momi("./100_folded/dadi/pop1-pop2.sfs")
dadi_to_momi("./100_unfolded/dadi/pop1-pop2.sfs")


/home/isaac/easySFS/jupyter-notebooks
Processing 2 populations - ['pop1', 'pop2']
  Sampling one snp per locus
Doing 1D sfs - pop1
Doing 1D sfs - pop2
Doing 2D sfs - ('pop1', 'pop2')
Doing multiSFS for all pops
Processing 2 populations - ['pop1', 'pop2']
  Sampling one snp per locus
Doing 1D sfs - pop1
Doing 1D sfs - pop2
Doing 2D sfs - ('pop1', 'pop2')
Doing multiSFS for all pops
momi sfs conversion requires unfolded sfs. Folded sfs will 'work' but with unknown results.


In [173]:
!pwd
!../easySFS.py -i 1000loci.vcf -p simout/simpops.txt --proj 8,8 -o 1000_folded -f
!../easySFS.py -i 1000loci.vcf -p simout/simpops.txt --proj 8,8 -o 1000_unfolded --unfolded -f
dadi_to_momi("./1000_folded/dadi/pop1-pop2.sfs")
dadi_to_momi("./1000_unfolded/dadi/pop1-pop2.sfs")


/home/isaac/easySFS/jupyter-notebooks
Processing 2 populations - ['pop1', 'pop2']
  Sampling one snp per locus
Doing 1D sfs - pop1
Doing 1D sfs - pop2
Doing 2D sfs - ('pop1', 'pop2')
Doing multiSFS for all pops
Processing 2 populations - ['pop1', 'pop2']
  Sampling one snp per locus
Doing 1D sfs - pop1
Doing 1D sfs - pop2
Doing 2D sfs - ('pop1', 'pop2')
Doing multiSFS for all pops
momi sfs conversion requires unfolded sfs. Folded sfs will 'work' but with unknown results.
